In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [2]:
link = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"
df_titanic = pd.read_csv(link)
df_titanic.head()

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250
3,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35.0,1,0,53.1000
4,0,3,Mr. William Henry Allen,male,35.0,0,0,8.0500


Data prep

In [3]:
#On transforme les valeurs de la colonne 'survived' pour plus de clareté : 1 = Survived , 0 = Dead
df_titanic['Survived'] = df_titanic['Survived'].apply(lambda x: "Survived" if x == 1 else "Dead")

In [4]:
#On factorise la colonne sex
df_titanic['Sex'] = df_titanic['Sex'].factorize()[0]

1. Tu vas effectuer une classification supervisée sur la colonne "survived", en utilisant le classifieur decision tree. Tu dois faire apparaitre clairement l'accuracy score du jeu de test.

In [11]:
X = df_titanic[['Pclass' ,'Sex',	'Age' ,	'Siblings/Spouses Aboard' ,	'Parents/Children Aboard' ,	'Fare']]
y = df_titanic['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y)

model = DecisionTreeClassifier().fit(X_train,y_train)

print("\nScore for the Train dataset :", model.score(X_train, y_train))
print("Score for the Test dataset :", model.score(X_test, y_test))


Score for the Train dataset : 0.9849624060150376
Score for the Test dataset : 0.7657657657657657


In [ ]:
#Pas mal d'overfitting, il va donc être nécessaire de jouer avec les paramètres pour avoir un meilleur modèle

2. Avec ce même classifieur, tu vas lancer une GridSearch avec les valeurs :

   - tous les entiers de 1 à 50 du paramètre max_depth
   - tous les entiers entre 1 et 15 du paramètre min_samples_leaf
   - (2, 5, 7, 10, 15, 30) du paramètre min_samples_split


In [13]:
#pour se rappeler des différents paramètres du Decision Tree
DecisionTreeClassifier().get_params().keys()

dict_keys(['ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [16]:
dico = {'max_depth' : range(1,50), 
        'min_samples_leaf' : range(1,15),
        'min_samples_split' : [2, 5, 7, 10, 15, 30]}


In [17]:
grid = GridSearchCV(DecisionTreeClassifier(), dico)

grid.fit(X,y)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 50),
                         'min_samples_leaf': range(1, 15),
                         'min_samples_split': [2, 5, 7, 10, 15, 30]})

3. Avec cette recherche via GridSearch, quelles sont le meilleures valeurs des hyperparamètres si on cherche à maximiser l'accuracy score ?

In [15]:
print("best score:",grid.best_score_)

print("best parameters:",grid.best_params_)

best score: 0.829772106900273
best parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2}


4. Avec ce même classifieur, tu vas lancer un RandomSearch sur les mêmes valeurs des paramètres en limitant à 200 itérations.

In [19]:
rando = RandomizedSearchCV(DecisionTreeClassifier(), dico , n_iter=200)

rando.fit(X,y)

RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=200,
                   param_distributions={'max_depth': range(1, 50),
                                        'min_samples_leaf': range(1, 15),
                                        'min_samples_split': [2, 5, 7, 10, 15,
                                                              30]})

5. Avec cette recherche via RandomSearch, quelles sont le meilleures valeurs des hyperparamètres si on cherche à maximiser l'accuracy score ?

In [20]:
print("best score:",rando.best_score_)

print("best parameters:",rando.best_params_)

best score: 0.8241668253665969
best parameters: {'min_samples_split': 5, 'min_samples_leaf': 7, 'max_depth': 8}


In [21]:
# Les meilleurs paramètres trouvés par les 2 méthodes sont différents, puisque le Randomized Search ne testent pas toutes les valeurs. 
# Cependant les meilleurs scores trouvés sont très proches. On peut donc utiliser l'une ou l'autre des combinaisons et nos résultats 
# devraient être très proches.